# ASA × LFM2.5 — Liquid Prompt τ/α Tuning

**목표**: Liquid 공식 포맷에서 ASA 억제가 너무 강한 문제(Recall 0.63→0.21)를 해결.
- 기존: τ=0.50, α=1 → FPR **0.116** (최저) but Recall **0.213** (폭락)
- 논문: τ=0.60, α=4 → 더 보수적 게이트 + 더 강한 스티어링

**핵심 아이디어**: τ를 높이면 "확실할 때만" 개입하고, α를 키우면 개입 시 더 효과적.

## 실험 매트릭스

| 실험 | τ | α | 전략 |
|------|---|---|------|
| baseline | 0.50 | 1 | 현재 (비교 기준) |
| paper | **0.60** | **4** | 논문 설정 그대로 |
| conservative | **0.70** | 3 | 보수적 게이트 + 중간 강도 |
| balanced | **0.65** | 5 | 적당한 게이트 + 강한 촉진 |
| gentle | **0.75** | 2 | 약한 개입 |
| aggressive | 0.55 | 8 | 약간만 완화 + 매우 강한 촉진 |

## 1. Setup

In [ ]:
!pip install -q transformers accelerate scikit-learn datasets tqdm matplotlib seaborn

import json, re, pickle, os, sys, warnings, gc, ast
import numpy as np
import torch
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from tqdm.auto import tqdm
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
print(f"PyTorch {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load Data (Same Pipeline as LiquidPrompt notebook)

In [ ]:
from datasets import load_dataset

ds = load_dataset("tatsu-lab/alpaca", split="train")
print(f"Alpaca: {len(ds)} samples")

# Tool definitions
TOOLS = [
    {"name": "calculator", "description": "Evaluate a mathematical expression and return the numeric result.",
     "parameters": {"type": "object", "properties": {"expression": {"type": "string", "description": "Math expression"}}, "required": ["expression"]}},
    {"name": "python_interpreter", "description": "Execute Python code and return the output.",
     "parameters": {"type": "object", "properties": {"code": {"type": "string", "description": "Python source code"}}, "required": ["code"]}},
    {"name": "web_search", "description": "Search the web for up-to-date information.",
     "parameters": {"type": "object", "properties": {"query": {"type": "string", "description": "Search query"}}, "required": ["query"]}},
    {"name": "translator", "description": "Translate text from one language to another.",
     "parameters": {"type": "object", "properties": {"text": {"type": "string", "description": "Text to translate"}, "target_language": {"type": "string", "description": "Target language"}}, "required": ["text", "target_language"]}}
]

# Liquid official system prompt
SYS_PROMPT = f"List of tools: {json.dumps(TOOLS)}"

# Keyword filters
KW = {
    "math": {"calculate", "compute", "solve", "equation", "sum", "average",
             "percentage", "convert", "ratio", "divide", "multiply"},
    "code": {"write a program", "code", "function", "algorithm", "implement",
             "script", "debug", "compile", "class", "method"},
    "search": {"search", "find", "look up", "latest", "current", "news",
               "recent", "who is", "what happened", "when did"},
    "translation": {"translate", "translation", "say in", "convert to",
                    "how do you say", "in french", "in spanish", "in german",
                    "in japanese", "in chinese"},
}

def classify(text):
    t = text.lower()
    for domain, keywords in KW.items():
        if any(k in t for k in keywords):
            return domain
    return None

np.random.seed(42)
buckets = {"math": {"tool": [], "non_tool": []}, "code": {"tool": [], "non_tool": []},
           "search": {"tool": [], "non_tool": []}, "translation": {"tool": [], "non_tool": []}}

for i, row in enumerate(ds):
    text = row.get("instruction", "") + " " + row.get("input", "")
    dom = classify(text)
    if dom is None:
        continue
    output = row.get("output", "")
    label = "tool" if classify(output) == dom or any(k in output.lower() for k in ["result", "output", "answer", "return"]) else "non_tool"
    buckets[dom][label].append(i)

for dom in buckets:
    for lbl in buckets[dom]:
        np.random.shuffle(buckets[dom][lbl])

PER_SPLIT_PER_CLASS = 40
SPLITS = {"cal": 0, "train": 1, "valid": 2, "test": 3}
TEST_MULT = 2

samples = {s: [] for s in SPLITS}
used = set()
for dom in buckets:
    for lbl in ["tool", "non_tool"]:
        pool = [i for i in buckets[dom][lbl] if i not in used]
        for sname, sidx in SPLITS.items():
            n = PER_SPLIT_PER_CLASS * (TEST_MULT if sname == "test" else 1)
            start = sidx * PER_SPLIT_PER_CLASS
            end = start + n
            chosen = pool[start:end]
            used.update(chosen)
            is_tool = 1 if lbl == "tool" else 0
            for idx in chosen:
                row = ds[int(idx)]
                text = row.get("instruction", "") + " " + row.get("input", "")
                samples[sname].append({"text": text, "label": is_tool, "domain": dom, "idx": int(idx)})

for s in samples:
    np.random.shuffle(samples[s])

print("\nSplit sizes:")
for s in samples:
    t = sum(1 for x in samples[s] if x["label"] == 1)
    nt = len(samples[s]) - t
    print(f"  {s:5s}: {len(samples[s])} ({t} tool / {nt} non-tool)")
print(f"Total: {sum(len(v) for v in samples.values())} unique samples")

## 3. Load Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "LiquidAI/LFM2.5-1.2B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True
)
model.eval()

TOOL_S = "<|tool_call_start|>"
TOOL_E = "<|tool_call_end|>"
NUM_LAYERS = model.config.num_hidden_layers
print(f"{MODEL_ID} ({sum(p.numel() for p in model.parameters()) // 1_000_000}M params, {NUM_LAYERS} layers)")

## 4. Load Saved Assets from LiquidPrompt Experiment

In [ ]:
# Check if assets exist from previous run, otherwise extract fresh
ASSET_DIR = Path("outputs_liquid/asa_assets")

if ASSET_DIR.exists():
    print("Loading saved assets from outputs_liquid/asa_assets/")
    import pickle
    router = pickle.load(open(ASSET_DIR / "router.pkl", "rb"))
    probes = pickle.load(open(ASSET_DIR / "probes.pkl", "rb"))
    scaler = pickle.load(open(ASSET_DIR / "scaler.pkl", "rb"))
    config = json.load(open(ASSET_DIR / "config.json"))
    vecs_npz = np.load(ASSET_DIR / "steering_vectors.npz")
    steering_vecs = {k: vecs_npz[k] for k in vecs_npz.files}
    L_STAR = config["L_star"]
    DOMAINS = config["domains"]
    print(f"  L* = {L_STAR}, domains = {DOMAINS}")
    print(f"  Original: alpha={config['alpha']}, tau={config['tau']}, beta={config['beta']}")
else:
    raise FileNotFoundError(
        "outputs_liquid/asa_assets/ not found! "
        "Run ASA_LFM25_LiquidPrompt.ipynb first to generate assets."
    )

## 5. Define Generation & Evaluation Functions

In [ ]:
def generate(messages, hook_fn=None, layer=None, max_new=128):
    '''Generate text, optionally with ASA hook.'''
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
    hook = None
    if hook_fn and layer is not None:
        hook = model.model.layers[layer].register_forward_hook(hook_fn)
    with torch.no_grad():
        out = model.generate(ids, max_new_tokens=max_new, do_sample=False,
                             pad_token_id=tokenizer.eos_token_id)
    if hook:
        hook.remove()
    return tokenizer.decode(out[0][ids.shape[1]:], skip_special_tokens=False)

def evaluate_split(split_data, alpha, tau, beta, use_asa=True):
    '''Evaluate a split with given hyperparameters.'''
    global _injected

    # Build mixed steering vector based on beta
    def get_steer_vec(domain):
        v_d = steering_vecs.get(domain)
        v_g = steering_vecs.get("global")
        if v_d is None:
            return v_g
        if v_g is not None and beta > 0:
            v = beta * v_g + (1 - beta) * v_d
        else:
            v = v_d
        return v / (np.linalg.norm(v) + 1e-8)

    def asa_hook(module, inp, out):
        global _injected
        if _injected:
            return out
        _injected = True
        h = out[0] if isinstance(out, tuple) else out
        hl = h[:, -1, :].detach().cpu().float().numpy()
        hs = scaler.transform(hl)
        dom = DOMAINS[router.predict(hs)[0]]
        pt = probes[dom].predict_proba(hs)[0, 1] if dom in probes else 0.5
        gate = 1 if pt >= tau else (-1 if pt <= 1 - tau else 0)
        if gate == 0:
            return out
        v = get_steer_vec(dom)
        vt = torch.tensor(v, dtype=torch.float16).to(h.device)
        hn = h.clone()
        hn[:, -1, :] += gate * alpha * vt
        rest = out[1:] if isinstance(out, tuple) else None
        return (hn,) + rest if rest else hn

    y_true, y_pred = [], []
    json_ok, schema_ok, args_ok = 0, 0, 0
    n_triggered = 0
    domain_results = {}

    for sample in tqdm(split_data, desc=f"Eval a={alpha} t={tau}", leave=False):
        text = sample["text"]
        label = sample["label"]
        domain = sample["domain"]
        msgs = [{"role": "system", "content": SYS_PROMPT},
                {"role": "user", "content": text}]

        if use_asa:
            _injected = False
            out = generate(msgs, hook_fn=asa_hook, layer=L_STAR)
        else:
            out = generate(msgs)

        triggered = 1 if TOOL_S in out else 0
        y_true.append(label)
        y_pred.append(triggered)

        if triggered:
            n_triggered += 1
            tc = out.split(TOOL_S)[-1].split(TOOL_E)[0].strip() if TOOL_E in out else ""
            try:
                parsed = json.loads(tc)
                json_ok += 1
                if isinstance(parsed, dict) and "name" in parsed:
                    schema_ok += 1
                    if "arguments" in parsed or "parameters" in parsed:
                        args_ok += 1
            except:
                try:
                    parsed = ast.literal_eval(tc)
                    if isinstance(parsed, (dict, list)):
                        json_ok += 1
                except:
                    pass

        if domain not in domain_results:
            domain_results[domain] = {"y_true": [], "y_pred": []}
        domain_results[domain]["y_true"].append(label)
        domain_results[domain]["y_pred"].append(triggered)

    # Overall metrics
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    results = {
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "fpr": fp / (fp + tn) if (fp + tn) > 0 else 0,
        "accuracy": accuracy_score(y_true, y_pred),
        "json_valid": json_ok / n_triggered if n_triggered > 0 else 0,
        "schema_ok": schema_ok / n_triggered if n_triggered > 0 else 0,
        "args_ok": args_ok / n_triggered if n_triggered > 0 else 0,
    }

    # Per-domain metrics
    for dom, dr in domain_results.items():
        tn_d, fp_d, fn_d, tp_d = confusion_matrix(dr["y_true"], dr["y_pred"], labels=[0, 1]).ravel()
        results[f"{dom}_f1"] = f1_score(dr["y_true"], dr["y_pred"], zero_division=0)
        results[f"{dom}_precision"] = precision_score(dr["y_true"], dr["y_pred"], zero_division=0)
        results[f"{dom}_recall"] = recall_score(dr["y_true"], dr["y_pred"], zero_division=0)
        results[f"{dom}_fpr"] = fp_d / (fp_d + tn_d) if (fp_d + tn_d) > 0 else 0

    return results

# Verify hook works
_injected = False
print("Evaluation functions defined.")

## 6. Baseline (No ASA)

In [ ]:
print("Evaluating Baseline (no ASA) on TEST set...")
baseline_results = evaluate_split(samples["test"], alpha=0, tau=0, beta=0, use_asa=False)

print(f"\n{'='*60}")
print(f"[Baseline - Liquid Prompt, No ASA]")
print(f"  Trig P/R/F1: {baseline_results['precision']:.4f} / {baseline_results['recall']:.4f} / {baseline_results['f1']:.4f}")
print(f"  FPR: {baseline_results['fpr']:.4f}  Acc: {baseline_results['accuracy']:.4f}")
print(f"{'='*60}")

## 7. τ/α Grid Sweep (핵심 실험)

> **논문 참고**: ASA 논문에서 Qwen2.5-1.5B는 α=4, τ=0.60 사용.
> 우리 기존 결과는 α=1, τ=0.50 → 과잉 억제 문제.
> τ를 높이면 "확실할 때만" 개입, α를 키우면 개입 시 더 효과적.

In [ ]:
# Experiment configurations
EXPERIMENTS = [
    # (name, tau, alpha, beta)
    ("current",      0.50, 1, 0.0),   # 현재 설정 (비교 기준)
    ("paper",        0.60, 4, 0.0),   # 논문 설정 그대로
    ("gentle",       0.70, 2, 0.0),   # 부드러운 개입
    ("balanced",     0.65, 3, 0.0),   # 균형잡힌 설정
    ("conservative", 0.70, 4, 0.0),   # 보수적 + 논문 α
    ("selective",    0.75, 5, 0.0),   # 매우 선택적 + 강한 촉진
    ("paper_high",   0.60, 8, 0.0),   # 논문 τ + 매우 강한 α
    ("minimal",      0.80, 3, 0.0),   # 최소 개입
]

all_results = {}
for name, tau, alpha, beta in EXPERIMENTS:
    print(f"\n{'='*60}")
    print(f"  Experiment: {name} (τ={tau}, α={alpha}, β={beta})")
    print(f"{'='*60}")
    results = evaluate_split(samples["test"], alpha=alpha, tau=tau, beta=beta, use_asa=True)
    all_results[name] = {"tau": tau, "alpha": alpha, "beta": beta, **results}
    print(f"  F1={results['f1']:.4f}  Prec={results['precision']:.4f}  "
          f"Rec={results['recall']:.4f}  FPR={results['fpr']:.4f}  Acc={results['accuracy']:.4f}")

print("\n\nAll experiments complete!")

## 8. Results Comparison Table

In [ ]:
print("=" * 90)
print(f"{'Experiment':>14s} | {'τ':>4s} | {'α':>3s} | {'F1':>6s} | {'Prec':>6s} | {'Rec':>6s} | {'FPR':>6s} | {'Acc':>6s}")
print("-" * 90)

# Baseline row
print(f"{'baseline':>14s} | {'--':>4s} | {'--':>3s} | {baseline_results['f1']:>6.4f} | "
      f"{baseline_results['precision']:>6.4f} | {baseline_results['recall']:>6.4f} | "
      f"{baseline_results['fpr']:>6.4f} | {baseline_results['accuracy']:>6.4f}")
print("-" * 90)

# ASA experiments sorted by F1
sorted_results = sorted(all_results.items(), key=lambda x: x[1]['f1'], reverse=True)
for name, r in sorted_results:
    marker = " ★" if r['f1'] == max(v['f1'] for v in all_results.values()) else ""
    print(f"{name:>14s} | {r['tau']:>4.2f} | {r['alpha']:>3d} | {r['f1']:>6.4f} | "
          f"{r['precision']:>6.4f} | {r['recall']:>6.4f} | {r['fpr']:>6.4f} | "
          f"{r['accuracy']:>6.4f}{marker}")

print("=" * 90)

# Find best
best_f1 = max(all_results.items(), key=lambda x: x[1]['f1'])
best_fpr = min(all_results.items(), key=lambda x: x[1]['fpr'])
best_recall = max(all_results.items(), key=lambda x: x[1]['recall'])

print(f"\n★ Best F1:     {best_f1[0]} (τ={best_f1[1]['tau']}, α={best_f1[1]['alpha']}) → F1={best_f1[1]['f1']:.4f}")
print(f"★ Best FPR:    {best_fpr[0]} (τ={best_fpr[1]['tau']}, α={best_fpr[1]['alpha']}) → FPR={best_fpr[1]['fpr']:.4f}")
print(f"★ Best Recall: {best_recall[0]} (τ={best_recall[1]['tau']}, α={best_recall[1]['alpha']}) → Rec={best_recall[1]['recall']:.4f}")

## 9. Comparison with Qwen-style Best (F1=0.6185)

In [ ]:
QWEN_STYLE_BEST = {"f1": 0.6185, "precision": 0.7591, "recall": 0.5219,
                    "fpr": 0.1656, "accuracy": 0.6781}

print("=" * 80)
print(f"{'':>14s} | {'F1':>6s} | {'Prec':>6s} | {'Rec':>6s} | {'FPR':>6s} | {'Acc':>6s}")
print("-" * 80)
print(f"{'Qwen+ASA':>14s} | {QWEN_STYLE_BEST['f1']:>6.4f} | {QWEN_STYLE_BEST['precision']:>6.4f} | "
      f"{QWEN_STYLE_BEST['recall']:>6.4f} | {QWEN_STYLE_BEST['fpr']:>6.4f} | {QWEN_STYLE_BEST['accuracy']:>6.4f}")
print("-" * 80)

for name, r in sorted_results:
    delta_f1 = r['f1'] - QWEN_STYLE_BEST['f1']
    delta_fpr = r['fpr'] - QWEN_STYLE_BEST['fpr']
    f1_marker = "↑" if delta_f1 > 0 else "↓"
    fpr_marker = "↓✅" if delta_fpr < 0 else "↑❌"
    print(f"{'Liq+' + name:>14s} | {r['f1']:>6.4f} | {r['precision']:>6.4f} | "
          f"{r['recall']:>6.4f} | {r['fpr']:>6.4f} | {r['accuracy']:>6.4f}  "
          f"ΔF1={delta_f1:+.4f}{f1_marker}  ΔFPR={delta_fpr:+.4f}{fpr_marker}")

print("=" * 80)

# Check if any Liquid config beats Qwen-style
any_beats = any(r['f1'] > QWEN_STYLE_BEST['f1'] for r in all_results.values())
if any_beats:
    winner = max(all_results.items(), key=lambda x: x[1]['f1'])
    print(f"\n🎉 Liquid + {winner[0]} BEATS Qwen-style! F1={winner[1]['f1']:.4f} > {QWEN_STYLE_BEST['f1']:.4f}")
else:
    closest = min(all_results.items(), key=lambda x: abs(x[1]['f1'] - QWEN_STYLE_BEST['f1']))
    gap = QWEN_STYLE_BEST['f1'] - closest[1]['f1']
    print(f"\n⚠️ No Liquid config beats Qwen-style. Closest: {closest[0]} (gap={gap:.4f})")

## 10. Domain-level Analysis for Best Config

In [ ]:
best_name = max(all_results.items(), key=lambda x: x[1]['f1'])[0]
best = all_results[best_name]
print(f"Best config: {best_name} (τ={best['tau']}, α={best['alpha']})\n")

domains = ["math", "code", "search", "translation"]
print(f"{'Domain':>12s} | {'F1':>6s} | {'Prec':>6s} | {'Rec':>6s} | {'FPR':>6s}")
print("-" * 55)
for dom in domains:
    f1 = best.get(f"{dom}_f1", 0)
    p = best.get(f"{dom}_precision", 0)
    r = best.get(f"{dom}_recall", 0)
    fpr = best.get(f"{dom}_fpr", 0)
    print(f"{dom:>12s} | {f1:>6.4f} | {p:>6.4f} | {r:>6.4f} | {fpr:>6.4f}")

## 11. Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

names = [n for n, _ in sorted_results]
f1s = [r['f1'] for _, r in sorted_results]
fprs = [r['fpr'] for _, r in sorted_results]
recalls = [r['recall'] for _, r in sorted_results]

# F1 bar chart
ax = axes[0]
colors = ['#2ecc71' if f > QWEN_STYLE_BEST['f1'] else '#e74c3c' if f < 0.4 else '#3498db' for f in f1s]
bars = ax.barh(names, f1s, color=colors)
ax.axvline(QWEN_STYLE_BEST['f1'], color='orange', linestyle='--', label=f"Qwen+ASA ({QWEN_STYLE_BEST['f1']:.3f})")
ax.axvline(baseline_results['f1'], color='gray', linestyle=':', label=f"Baseline ({baseline_results['f1']:.3f})")
ax.set_xlabel('F1 Score')
ax.set_title('F1 Score by Config')
ax.legend(fontsize=8)
ax.set_xlim(0, 1)

# FPR bar chart
ax = axes[1]
colors = ['#2ecc71' if f < QWEN_STYLE_BEST['fpr'] else '#e74c3c' if f > 0.3 else '#3498db' for f in fprs]
bars = ax.barh(names, fprs, color=colors)
ax.axvline(QWEN_STYLE_BEST['fpr'], color='orange', linestyle='--', label=f"Qwen+ASA ({QWEN_STYLE_BEST['fpr']:.3f})")
ax.axvline(baseline_results['fpr'], color='gray', linestyle=':', label=f"Baseline ({baseline_results['fpr']:.3f})")
ax.set_xlabel('FPR (lower=better)')
ax.set_title('False Positive Rate by Config')
ax.legend(fontsize=8)
ax.set_xlim(0, 0.8)

# Recall bar chart
ax = axes[2]
colors = ['#2ecc71' if r > 0.4 else '#e74c3c' if r < 0.25 else '#3498db' for r in recalls]
bars = ax.barh(names, recalls, color=colors)
ax.axvline(QWEN_STYLE_BEST['recall'], color='orange', linestyle='--', label=f"Qwen+ASA ({QWEN_STYLE_BEST['recall']:.3f})")
ax.axvline(baseline_results['recall'], color='gray', linestyle=':', label=f"Baseline ({baseline_results['recall']:.3f})")
ax.set_xlabel('Recall')
ax.set_title('Recall by Config')
ax.legend(fontsize=8)
ax.set_xlim(0, 1)

plt.suptitle('ASA τ/α Tuning — Liquid Prompt on LFM2.5', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("outputs_liquid/tuning_comparison.png", dpi=150, bbox_inches='tight')
plt.show()
print("Chart saved to outputs_liquid/tuning_comparison.png")

## 12. FPR vs Recall Pareto Front

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Plot all experiments
for name, r in all_results.items():
    ax.scatter(r['fpr'], r['recall'], s=100, zorder=5)
    ax.annotate(f"{name}\nτ={r['tau']},α={r['alpha']}\nF1={r['f1']:.3f}",
                (r['fpr'], r['recall']), textcoords="offset points",
                xytext=(10, 5), fontsize=8)

# Reference points
ax.scatter(baseline_results['fpr'], baseline_results['recall'], s=200, marker='X',
           color='gray', zorder=6, label='Baseline (no ASA)')
ax.scatter(QWEN_STYLE_BEST['fpr'], QWEN_STYLE_BEST['recall'], s=200, marker='D',
           color='orange', zorder=6, label='Qwen+ASA (best)')

# Ideal region
ax.axhspan(0.4, 1.0, xmin=0, xmax=0.3, alpha=0.1, color='green', label='Ideal zone')

ax.set_xlabel('FPR (lower = better) →', fontsize=12)
ax.set_ylabel('Recall (higher = better) →', fontsize=12)
ax.set_title('FPR vs Recall Tradeoff — Liquid Prompt ASA Tuning', fontsize=14)
ax.legend(loc='upper right')
ax.set_xlim(-0.02, 0.75)
ax.set_ylim(-0.02, 0.75)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs_liquid/pareto_front.png", dpi=150, bbox_inches='tight')
plt.show()
print("Pareto chart saved to outputs_liquid/pareto_front.png")

## 13. Final Summary & Recommendation

In [ ]:
print("=" * 70)
print("  FINAL SUMMARY: Liquid Prompt τ/α Tuning")
print("=" * 70)

best_name, best = max(all_results.items(), key=lambda x: x[1]['f1'])
best_fpr_name, best_fpr_r = min(all_results.items(), key=lambda x: x[1]['fpr'])

print(f"\n  [Reference] Qwen-style + ASA:")
print(f"    F1={QWEN_STYLE_BEST['f1']:.4f}  FPR={QWEN_STYLE_BEST['fpr']:.4f}  Rec={QWEN_STYLE_BEST['recall']:.4f}")

print(f"\n  [Best F1] Liquid + {best_name} (τ={best['tau']}, α={best['alpha']}):")
print(f"    F1={best['f1']:.4f}  FPR={best['fpr']:.4f}  Rec={best['recall']:.4f}")

print(f"\n  [Best FPR] Liquid + {best_fpr_name} (τ={best_fpr_r['tau']}, α={best_fpr_r['alpha']}):")
print(f"    F1={best_fpr_r['f1']:.4f}  FPR={best_fpr_r['fpr']:.4f}  Rec={best_fpr_r['recall']:.4f}")

delta_f1 = best['f1'] - QWEN_STYLE_BEST['f1']
if delta_f1 > 0:
    print(f"\n  🎉 RESULT: Liquid prompt with tuned τ/α BEATS Qwen-style by +{delta_f1:.4f} F1!")
    print(f"  → Recommendation: Use Liquid prompt with τ={best['tau']}, α={best['alpha']}")
elif delta_f1 > -0.05:
    print(f"\n  ⚠️ RESULT: Close to Qwen-style (gap={-delta_f1:.4f}) with better FPR.")
    if best['fpr'] < QWEN_STYLE_BEST['fpr']:
        print(f"  → Liquid has lower FPR ({best['fpr']:.4f} vs {QWEN_STYLE_BEST['fpr']:.4f})")
        print(f"  → Consider Liquid for FPR-sensitive applications")
else:
    print(f"\n  ❌ RESULT: Qwen-style remains better by {-delta_f1:.4f} F1.")
    print(f"  → Stick with Qwen-style + ASA for best overall performance")
    if best_fpr_r['fpr'] < QWEN_STYLE_BEST['fpr']:
        print(f"  → But Liquid + {best_fpr_name} is best for FPR-critical scenarios ({best_fpr_r['fpr']:.4f})")

print("=" * 70)

## 14. Save Results

In [ ]:
# Save all results
save_data = {
    "baseline": baseline_results,
    "qwen_style_reference": QWEN_STYLE_BEST,
    "experiments": all_results,
    "best_f1_config": best_name,
    "best_fpr_config": best_fpr_name,
}

os.makedirs("outputs_liquid", exist_ok=True)
with open("outputs_liquid/tuning_results.json", "w") as f:
    json.dump(save_data, f, indent=2, default=str)

print("Results saved to outputs_liquid/tuning_results.json")
print(f"\nTotal experiments run: {len(all_results)}")
print(f"Best F1: {best_name} → {best['f1']:.4f}")
print(f"Best FPR: {best_fpr_name} → {best_fpr_r['fpr']:.4f}")